This is a document I use to separate out functions of the main script in order to troubleshoot or quickly bypass some steps. I often use this to avoid spamming chat or API requests when I want to test changes in a script without using quota credits or some other resource.

In [1]:
from funcs import *
import requests,time
from pprint import pprint
showbot_channel,token = 'Frogpants','Nhrj6amcz4AqiSP6AVv5YhhQX8OhJ6wO'

In [2]:
streamid = find_stream_id('@ScottJohnson')
if streamid:
    print(f'Live stream detected.\nLive stream id: {streamid}')
else:
    print('No live stream found.')

Live stream detected.
Live stream id: QvKqwCpdqmU


In [2]:
streamid = 'QvKqwCpdqmU'

For testing

In [3]:
authors,messages = parse_chat(read_chat(find_chat_id(streamid)))
for author,message in zip(authors,messages):
    print(f'{author}: {message}')

KeyError: 'activeLiveChatId'

For titles

In [ ]:
author_index,titles = title_search(read_chat(find_chat_id(streamid)))
# for author,title in zip(authors,titles):
#     print(f'{author}: {title}')

In [18]:
authors_html,titles_html = [html_ify(auth) for auth in author_index],[html_ify(title) for title in titles]
for author_html,title_html in zip(authors_html,titles_html):
    print(f'{author_html}: {title_html}')
    

In [4]:
livechatid = find_chat_id(streamid)

KeyError: 'activeLiveChatId'

In [4]:
livechatid


'Cg0KC0tqdjV5bWsxTlpBKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILS2p2NXltazFOWkE'

In [12]:
livechatid = 'Cg0KC3Rhb0lYcU9Udi1jKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILdGFvSVhxT1R2LWM'

In [6]:
client_secrets_file

'../client_secrets_file.json'

In [9]:
author_index,submitted_titles = build_submission_history(showbot_channel)

Titles scraped for Showbot.tv channel: Frogpants 


In [10]:
for author,title in zip(authors,titles):
    print(f'{author}: {title}')

NameError: name 'authors' is not defined

In [5]:
credentials = Credentials.from_authorized_user_file("../working_submit_token_Talia_do_not_delete.json", scopes)
youtube = build_yt_obj(credentials,api_service_name, api_version,module=googleapiclient.discovery)

In [7]:
credentials = build_credentials(client_secrets_file,scopes,testing=True)
credentials

Experimental Token Used.


In [5]:
credentials = build_credentials(client_secrets_file,scopes)

In [6]:
youtube = build_yt_obj(credentials,api_service_name, api_version,module=googleapiclient.discovery)
youtube

In [16]:
request = youtube.liveChatMessages().list(
    liveChatId=livechatid,
    part="snippet,authorDetails"
)

In [38]:
streamid = 'a5sdsbGKz8U'

In [39]:
request = youtube.videos().list(
    part="sniffppet,contentDetails,statistics",
    id=streamid
)

In [37]:
request = youtube.liveChatMessages().insert(part="snippet",body={"snippet": {"liveChatId": livechatid,
        "type": "textMessageEvent",
        "textMessageDetails": {
            "messageText": 'Awesome.'}}})

NameError: name 'livechatid' is not defined

In [42]:
response = request.execute()


AttributeError: 'dict' object has no attribute 'status_code'

In [35]:
response['items']

[{'kind': 'youtube#video',
  'etag': '8W2wPWCalpyX1-VX34muGDA3wg8',
  'id': 'a5sdsbGKz8U',
  'snippet': {'publishedAt': '2024-10-07T16:03:16Z',
   'channelId': 'UCSJ4gkVC6NrvII8umztf0Ow',
   'title': 'Halloween lofi radio  🎃 - spooky beats to get chills to',
   'description': "🎼 | Listen on Spotify, Apple music and more\n→  https://fanlink.tv/Lofi-Girl-Halloween-2024\n\n🎃 Welcome to Lofi Girl's Halloween radio! We hope you have a spooktacular time here\n\n👻 Get your Halloween Vinyl \n→ https://bit.ly/4gu0KZN\n\n🌎 | Lofi Girl on all social media\n→  https://fanlink.tv/lofigirl-social\n\n👕 | Lofi Girl merch\n→  https://bit.ly/Iofigirl-shop\n\n🎭 | Create your lofi avatar now\n→  https://lofigirl.com/generator/\n\n💬 | Join the Lofi Girl community\n→   https://bit.ly/lofigirl-discord\n→   https://bit.ly/lofigirl-reddit\n\n🎶 | Radio tracklist\n→  https://bit.ly/lofi-full-tracklist\n\n🎨 | Art by Lofi Studio (full list of artists here)\n→  https://www.instagram.com/p/CuChqFXs08M/",
   'thumbna

In [27]:
response

{'kind': 'youtube#videoListResponse',
 'etag': 'YIUPVpqNjppyCWOZfL-19bLb7uk',
 'items': [],
 'pageInfo': {'totalResults': 0, 'resultsPerPage': 0}}

In [27]:
[i['etag'] for i in response['items']]

['WTSCnZPaeUrfLZr0Cvs1WYoEEOU',
 'nTDMGksUBkj_PPGWwZ8GajyBFAQ',
 'FqyU7sDmh8b49tjwETFoorjnnRw',
 '90cumXRqOaboNFeaUZfzG3uG5qU',
 'GA7-iWZT40P4OGzEAeVsBKiiZL0',
 'ThVLmOL8wxS_3J_1_ZgnI-bF98w',
 '_JTyPHwIYx3AW3Qk1LgRArVDn_4',
 'oHqR1hyaDfNF4fmFLv7MhJy2eh0',
 'jYzpyNnkHALZIMiIYlb9BwYIn7Q',
 'aGYy-YH7SXTlW0fiTb8IS3D02P0',
 '2zpGylHBnH8VHgc4zF2LpNpL3kw',
 '-slCZadxAygEGfv2Avk957uiNa8',
 'cp_uGmp5OKaymKGnEbho89WR7BY',
 '0zBtFiX_D2S31kYs6pbn-gt1M8U',
 'eBDDMOEqGjRQQ6lK3RNGeOytG1g',
 '1DomSo51A7sDvTPs7Xgi0JImN_I',
 'SfeztIqbjzYo2ltGVIO__cI2kt4',
 'ArTR1PCxKr4FCvZUOHWSUIIKsZ8',
 'tvrLZMZr5n7clppc-evNKrWwuMg',
 'uLFrVUCe772bAIxclFBogqVRa5c',
 'mKCdf4mTpsj9G2-MwisnY65PU6c',
 'QsIRjOLBn1db1NzeDMDWZb9muhY',
 'OKZNmIbJ_u9hocxWLW9Gluh3Qv0',
 'g5kv-0Qck3ieUxjDvtZyYkKwH7o',
 '5gQpmgBhW30pzk9MtHKxh8gc3Wk',
 'LN9Dev_07I2UY-Hk-EA3nLeHKHA',
 'v6y2VgBaj_HHZmEMon3b03LZyjk',
 'gunT-kJiekMhR7fankefu5iIEPI',
 'R2ztlrZjb86bVkI3gbDlw0Ppa9g',
 'FnTOH5JJJvfwykvz4h7yEAEimMc',
 'uUhn3_DezETOurc7WM6HrQFfX54',
 'es3fxc

In [12]:
int(response['pollingIntervalMillis']/1000)

KeyError: 'pollingIntervalMillis'

In [12]:
with open('../token.json','w') as token:
    token.write(credentials.to_json())

In [ ]:
os.getcwd()

In [6]:
confirmation_message = "[Bot] I fucking hate this."

In [ ]:
!gcloud auth list

In [12]:
request = youtube.liveChatMessages().insert(part="snippet",body={"snippet": {"liveChatId": livechatid,
        "type": "textMessageEvent",
        "textMessageDetails": {
            "messageText": confirmation_message}}})
response = request.execute()

In [7]:
send_chat_message('Cg0KC3Rhb0lYcU9Udi1jKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILdGFvSVhxT1R2LWM','Dicks')

Original Token Used.


HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/liveChat/messages?part=snippet&alt=json returned "The specified live chat is no longer live.". Details: "[{'message': 'The specified live chat is no longer live.', 'domain': 'youtube.liveChat', 'reason': 'liveChatEnded'}]">

In [9]:
try:
    send_chat_message('Cg0KC3Rhb0lYcU9Udi1jKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILdGFvSVhxT1R2LWM','Dicks')
except googleapiclient.errors.HttpError:
    print('No live chat detected, likely because the chat has ended.')

Original Token Used.
No live chat detected, likely because the chat has ended.


In [9]:
while True:
    authors,titles = title_search(read_chat(livechatid)) #Poll API for all chat messages, regardless of they've been polled already or not. title_search() already filters for '!s' chat trigger and cleans as needed
    #Check extracted titles versus previously submitted ones, submit if new
    for author,title in zip(authors,titles):
        if title not in submitted_titles: #Check extracted titles versus previously submitted ones, submit if new
            author_index.append(author)
            submitted_titles.append(title)
            print(f'{author}: {title}') #Add new title to list and add author to another list to maintain the order. Print for debugging and monitoring purposes.
            author_html,title_html = html_ify(author),html_ify(title) #Prepare link for submission then submit via Requests module
            submission_link = f'http://www.showbot.tv/s/add.php?title={title_html}&user={author_html}&channel={showbot_channel}&key={token}'
            requests.get(submission_link)
            #At this point the title has been submitted. This is one complete function and API call. Confirmation via the .insert() method to send a message is a separate 
            #function incurring higher quota useage.
            credentials = build_credentials(client_secrets_file,scopes) #Builds OAuth2 credentials object. Uses existing files or makes new one if not detected. This links the function to the account that follows the URI.
            youtube = build_yt_obj(googleapiclient.discovery,credentials) #Common step in calling the API. Always have to build an object on which to call methods.
            response = send_chat_message(livechatid,author) #The API response in JSON format. Not technically needed but saved in case.
            print(f'Title successfully submitted and confirmed: {author}') #Generic success message only for the console for debugging and monitoring purposes.
    time.sleep(5)

Titties
Talia James:  I have nothing snarky because this experience saps my will to live
Titties
Titties


HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/liveChat/messages?part=snippet&alt=json returned "Request contains an invalid argument.". Details: "[{'domain': 'youtube.api.v3.LiveChatMessageInsertResponse.Error', 'reason': 'ERROR_UNSPECIFIED'}]">

In [11]:
author_index,submitted_titles = build_submission_history(showbot_channel)

In [12]:
while True:
    authors,titles = title_search(read_chat(livechatid)) #Poll API for all chat messages, regardless of they've been polled already or not. title_search() already filters for '!s' chat trigger and cleans as needed
    #Check extracted titles versus previously submitted ones, submit if new
    for author,title in zip(authors,titles):
        if title not in submitted_titles: #Check extracted titles versus previously submitted ones, submit if new
            author_index.append(author)
            submitted_titles.append(title)
            print(f'{author}: {title}') #Add new title to list and add author to another list to maintain the order. Print for debugging and monitoring purposes.
            author_html,title_html = html_ify(author),html_ify(title) #Prepare link for submission then submit via Requests module
            submission_link = f'http://www.showbot.tv/s/add.php?title={title_html}&user={author_html}&channel={showbot_channel}&key={token}'
            try:
                requests.get(submission_link)
            except TimeoutError:
                print('Timeout error')
            #At this point the title has been submitted. This is one complete function and API call. Confirmation via the .insert() method to send a message is a separate 
            #function incurring higher quota useage.
            credentials = Credentials.from_authorized_user_file("../working_submit_token_Talia_do_not_delete.json", scopes)
            youtube = build_yt_obj(googleapiclient.discovery,credentials,api_service_name, api_version) #Common step in calling the API. Always have to build an object on which to call methods.
            try:
                response = send_chat_message(livechatid,author) #The API response in JSON format. Not technically needed but saved in case.
                print(f'Title successfully submitted and confirmed: {author}') #Generic success message only for the console for debugging and monitoring purposes.
            except googleapiclient.errors.HttpError: #For some reason the script can get a 403 response code error when attempting to submit a confirmation response. This block keeps the script from halting if that occurs.
                print(f'HTTP Error: [{author}: {title}] failed to submit.')# Trying again in 3 seconds.') #This is an experimental approach to handling the 403 error by attempting to resubmit the response asynchronously.
                # asyncio.sleep(3)
                # try:
                #     response = asyncio.send_async_chat_message(livechatid,author)
                # except googleapiclient.errors.HttpError:
                #     print(f'Submission of [{author}: {title}] failed.')
                # print(traceback.format_exc())
    time.sleep(5) 

Titties
Talia James:  I have nothing snarky because this experience saps my will to live
Titties
Submission attempted.
Title successfully submitted and confirmed: Talia James
Titties
Titties
Titties
Titties


KeyboardInterrupt: 